In [1]:
import pandas as pd
import numpy as np
import networkx as nx

In [70]:
data = pd.read_csv('train.csv')
data.head()
#data set here
# https://drive.google.com/drive/folders/1pVhFmDjzftveFKPwTI8PjvB46g3WhP-8?usp=sharing

,source_node,destination_node
0,1,690569
1,1,315892
2,1,189226
3,2,834328
4,2,1615927


In [71]:
len(data)

9437519

In [10]:
#taking first n points
takennodes = 1000000
#creating a directed graph
Graphtype = nx.DiGraph()
#add edges from a csv
G = nx.from_pandas_edgelist(data.head(takennodes), source = 'source_node', target = 'destination_node', create_using=Graphtype)

Use this to add one edge at a time
https://networkx.org/documentation/stable/reference/classes/generated/networkx.DiGraph.add_edge.html

YOu can add the corresponding edge to the undirected grapg which i am createing in next cell

In [11]:
curr_user =1
H = G.to_undirected()

In [68]:
def recommendor(Gr,curr_user,friends, k=3, alpha=0.7, rad = 3):
    connects = list(nx.ego_graph(Gr, curr_user, radius=rad, undirected=False).nodes)
    
    connects.remove(curr_user)
    connects = np.asarray(connects)
    node_pair = [(curr_user, node) for node in connects]
    users_score = np.zeros(len(node_pair))

#     CCAP = nx.common_neighbor_centrality(Gr,node_pair)
    AAI = nx.adamic_adar_index(Gr,node_pair)
#     CCAP_score = np.asarray([nodescore[2] for nodescore in CCAP])

    AAI_score = np.asarray([nodescore[2] for nodescore in AAI])

    final_score = AAI_score
    for i , node in enumerate(connects):
        final_score+= np.log(len(final_score))/nx.shortest_path_length(H, curr_user,node)
        if node in friends:
            final_score[i]=0
    if np.min(final_score)==np.max(final_score):
        return np.random.choice(connects,replace=False, size = k)
    # Insted of just choosing the top k best matches., let us sample from the distribution with the final scores as priors
    p = final_score - np.min(final_score) 
    p/=np.sum(p)
    return np.random.choice(connects, size = k,replace = False, p = p)    

In [69]:
userid = 1
friends = {n for n in G.neighbors(userid)}

# to get k recommendations for the user id = userid
recommendation = recommendor(Gr = H,curr_user = userid,k = 10, rad=4, friends=friends)

In [50]:
friends

{189226, 315892, 690569}